# Detailed Data Description of Rechunked National Water Model v2.1 Retrospective Simulation
This notebook gives a *detailed* look at the metadata of the six Zarr stores which comprise the rechunked NWM v2.1 retrospective simulation. 
The perspective of the metadata is both from the `xarray.Dataset` point of view (as xarray would work with the data in memory) and from the Zarr point of view as well (storage, compression). Details are given at the dataset/group level and also for each variable. Also shown for each is the chunking scheme of a single input file (model output file), for comparison.

For a less detailed data description please see this 
[(html)](https://htmlpreview.github.io/?https://github.com/NCAR/rechunk\_retro_nwm\_v21/blob/main/notebooks/data_description.html) 
[(notebook)](https://htmlpreview.github.io/?https://github.com/NCAR/rechunk\_retro_nwm\_v21/blob/main/notebooks/data_description.ipynb).

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import pathlib
import subprocess
import xarray as xr
import zarr as zr

/glade/work/jamesmcc/python_envs/379zr/lib/python3.7/site-packages/xarray/backends/cfgrib_.py:28: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  "Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. "


In [2]:
date = pd.to_datetime("1979-02-02")

input_dir = "/glade/scratch/zhangyx/WRF-Hydro/model.data.v2.1"
input_dir_precip = "/glade/scratch/jamesmcc/aorc_forcing_symlinks"
input_path = (
    f'{input_dir}/'
    f'{date.strftime("%Y")}/'
    f'{date.strftime("%Y%m%d%H%M")}')
input_path_precip = (
    f'{input_dir_precip}/'
    f'{date.strftime("%Y")}/'
    f'{date.strftime("%Y%m%d%H%M")}')

input_file_exs = {
    'chrtout': f'{input_path}.CHRTOUT_DOMAIN1.comp',
    'gwout': f'{input_path}.GWOUT_DOMAIN1.comp',
    'lakeout': f'{input_path}.LAKEOUT_DOMAIN1.comp',
    'ldasout': f'{input_path}.LDASOUT_DOMAIN1.comp',
    'rtout': f'{input_path}.RTOUT_DOMAIN1.comp',
    'precip': f'{input_path_precip}.LDASIN_DOMAIN1',
    }
assert all([pathlib.Path(ff).exists() for kk, ff in input_file_exs.items()])

Determine the output/zarr files. 

In [3]:
output_path_ishita = pathlib.Path("/glade/p/datashare/ishitas/nwm_retro_v2.1/")
output_path_james = pathlib.Path("/glade/p/datashare/jamesmcc/nwm_retro_v2.1/")

output_dict = {
    'lakeout': output_path_james / 'lakeout.zarr',
    'gwout': output_path_ishita / 'gwout.zarr',
    'chrtout': output_path_ishita / 'chrtout.zarr',
    'precip': output_path_james / 'precip.zarr',
    #'ldasout': output_path_ishita / 'ldasout.zarr',
    #'rtout': output_path_james / 'rtout.zarr',
    
}
assert all([ff.exists() for kk, ff in output_dict.items()])

This same report function will be applied to each dataset. 

In [4]:
def report(pattern):
    file_rechunked = output_dict[pattern]
    input_file_ex = input_file_exs[pattern]
    
    print(f"Output file report: {file_rechunked}")

    result = subprocess.run(
        f"du -shc {file_rechunked}", stdout=subprocess.PIPE, shell=True)
    file_size = result.stdout.decode("utf-8").split("\t")[0]
    print(f"Total file size: {file_size}")

    ds = xr.open_zarr(file_rechunked)
    dz = zr.open(file_rechunked)
    ds_src = xr.open_dataset(input_file_ex)
    
    print('')
    print("Dataset overviews:")
    
    print(f"xarray {pattern} SOURCE single file: ds.info")
    display(ds_src)
    
    print(f"xarray {pattern}: ds.info:")
    display(ds)

    print(f"zarr {pattern}: ds.info:")
    display(dz.info)
    
    print("Variable comparison xarray and zarr:")
    
    for vv in ds.variables:
        print(f"\n----------------------------------------------------------------------------------------------------------------")
        print(f"----------------------------------------------------------------------------------------------------------------\n")
        print(f"{pattern}: {vv}\n")
        
        if vv in ds_src.variables:
            print(f"xarray {pattern} SOURCE single file: ds[{vv}]:\n")
            display(ds_src[vv])
        else:
            print(f'{pattern}: {vv} not in source')        

        print(f"xarray {pattern}: ds[{vv}]:\n")
        display(ds[vv])

        print(f"")
        info = dz[vv].info
        print(f"zarr {pattern}:  dz[{vv}].info:")
        display(info)
        
        chunk_bytes = np.prod(info.obj.chunks) * info.obj.dtype.itemsize
        storage_ratio = float(str(info.obj._info_reporter).split(':')[-2].split('\n')[0].split(' ')[1])
        print(f"Un-Compressed Chunk size in MB: {chunk_bytes / 1048576}")
        print(f"   Compressed Chunk size in MB: {chunk_bytes / 1048576 / storage_ratio}")

    ds.close()
    ds_src.close()
        
    return None


## LAKEOUT: lake model output

In [5]:
report('lakeout')

Output file report: /glade/p/datashare/jamesmcc/nwm_retro_v2.1/lakeout.zarr
Total file size: 5.5G

Dataset overviews:
xarray lakeout SOURCE single file: ds.info


/glade/work/jamesmcc/python_envs/379zr/lib/python3.7/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.Dataset>
Dimensions:                      (time: 1, reference_time: 1, feature_id: 5783)
Coordinates:
  * time                         (time) datetime64[ns] 1979-02-02
  * reference_time               (reference_time) datetime64[ns] 1979-02-01
  * feature_id                   (feature_id) int32 491 531 ... 1021092845
    latitude                     (feature_id) float32 ...
    longitude                    (feature_id) float32 ...
Data variables:
    crs                          |S1 ...
    reservoir_type               (feature_id) int32 ...
    reservoir_assimilated_value  (feature_id) float32 ...
    water_sfc_elev               (feature_id) float32 ...
    inflow                       (feature_id) float64 ...
    outflow                      (feature_id) float64 ...
Attributes:
    TITLE:                      OUTPUT FROM WRF-Hydro v5.2.0-beta2
    featureType:                timeSeries
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    model_initialization_time:  1979-02-01_00:00:00
    station_dimension:          lake_id
    model_output_valid_time:    1979-02-02_00:00:00
    model_total_valid_times:    1416
    Conventions:                CF-1.6
    code_version:               v5.2.0-beta2
    model_output_type:          reservoir
    model_configuration:        retrospective

xarray lakeout: ds.info:


<xarray.Dataset>
Dimensions:         (feature_id: 5783, time: 367439)
Coordinates:
  * feature_id      (feature_id) int32 491 531 747 ... 947070204 1021092845
    latitude        (feature_id) float32 dask.array<chunksize=(5783,), meta=np.ndarray>
    longitude       (feature_id) float32 dask.array<chunksize=(5783,), meta=np.ndarray>
  * time            (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T...
Data variables:
    crs             |S1 ...
    inflow          (time, feature_id) float64 dask.array<chunksize=(8064, 500), meta=np.ndarray>
    outflow         (time, feature_id) float64 dask.array<chunksize=(8064, 500), meta=np.ndarray>
    water_sfc_elev  (time, feature_id) float32 dask.array<chunksize=(8064, 500), meta=np.ndarray>
Attributes:
    Conventions:                  CF-1.6
    TITLE:                        OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:                 v5.2.0-beta2
    featureType:                  timeSeries
    model_configuration:          retrospective
    model_output_type:            reservoir
    proj4:                        +proj=lcc +units=m +a=6370000.0 +b=6370000....
    reservoir_assimilated_value:  Assimilation not performed
    reservoir_type:               1 = level pool everywhere
    station_dimension:            lake_id

...

xarray lakeout: ds[water_sfc_elev]:



<xarray.DataArray 'water_sfc_elev' (time: 367439, feature_id: 5783)>
dask.array<open_dataset-fd8b16972b45e1dad0dd9bf62912480fwater_sfc_elev, shape=(367439, 5783), dtype=float32, chunksize=(8064, 500), chunktype=numpy.ndarray>
Coordinates:
  * feature_id  (feature_id) int32 491 531 747 ... 947070204 1021092845
    latitude    (feature_id) float32 dask.array<chunksize=(5783,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(5783,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Attributes:
    comment:    If reservoir_type = 4, water_sfc_elev is invalid because this...
    long_name:  Water Surface Elevation
    units:      m


zarr lakeout:  dz[water_sfc_elev].info:


Name,/water_sfc_elev
Type,zarr.core.Array
Data type,float32
Shape,"(367439, 5783)"
Chunk shape,"(8064, 500)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,8499598948 (7.9G)
No. bytes stored,2828358144 (2.6G)


Un-Compressed Chunk size in MB: 15.380859375
   Compressed Chunk size in MB: 5.126953125


## gwout file: groundwater model output

In [6]:
report('gwout')

Output file report: /glade/p/datashare/ishitas/nwm_retro_v2.1/gwout.zarr
Total file size: 1.7T

Dataset overviews:
xarray gwout SOURCE single file: ds.info


<xarray.Dataset>
Dimensions:         (time: 1, reference_time: 1, feature_id: 2776738)
Coordinates:
  * time            (time) datetime64[ns] 1979-02-02
  * reference_time  (reference_time) datetime64[ns] 1979-02-01
  * feature_id      (feature_id) int32 6635572 6635590 ... 25293410 15450136
Data variables:
    inflow          (feature_id) float64 ...
    outflow         (feature_id) float64 ...
    depth           (feature_id) float64 ...
Attributes:
    TITLE:                      OUTPUT FROM WRF-Hydro v5.2.0-beta2
    featureType:                timeSeries
    model_initialization_time:  1979-02-01_00:00:00
    station_dimension:          gw_id
    model_output_valid_time:    1979-02-02_00:00:00
    model_total_valid_times:    1416
    Conventions:                CF-1.6
    code_version:               v5.2.0-beta2
    model_output_type:          groundwater_rt
    model_configuration:        retrospective

xarray gwout: ds.info:


<xarray.Dataset>
Dimensions:     (time: 367439, feature_id: 2776738)
Coordinates:
  * feature_id  (feature_id) int32 6635572 6635590 6635598 ... 25293410 15450136
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Data variables:
    depth       (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    inflow      (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    outflow     (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
Attributes:
    Conventions:          CF-1.6
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    model_output_type:    groundwater_rt
    station_dimension:    gw_id

zarr gwout: ds.info:


...

xarray gwout: ds[time]:



<xarray.DataArray 'time' (time: 367439)>
array(['1979-02-01T01:00:00.000000000', '1979-02-01T02:00:00.000000000',
       '1979-02-01T03:00:00.000000000', ..., '2020-12-31T21:00:00.000000000',
       '2020-12-31T22:00:00.000000000', '2020-12-31T23:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:00:00
Attributes:
    long_name:      valid output time
    standard_name:  time
    valid_max:      4862880
    valid_min:      4777980


zarr gwout:  dz[time].info:


Name,/time
Type,zarr.core.Array
Data type,int64
Shape,"(367439,)"
Chunk shape,"(672,)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,2939512 (2.8M)
No. bytes stored,231171 (225.8K)


Un-Compressed Chunk size in MB: 0.005126953125
   Compressed Chunk size in MB: 0.00040369709645669296


## chrtout file: streamflow model output

In [7]:
report('chrtout')

Output file report: /glade/p/datashare/ishitas/nwm_retro_v2.1/chrtout.zarr
Total file size: 1.4T

Dataset overviews:
xarray chrtout SOURCE single file: ds.info


<xarray.Dataset>
Dimensions:         (time: 1, reference_time: 1, feature_id: 2776738)
Coordinates:
  * time            (time) datetime64[ns] 1979-02-02
  * reference_time  (reference_time) datetime64[ns] 1979-02-01
  * feature_id      (feature_id) int32 101 179 181 ... 1180001803 1180001804
    latitude        (feature_id) float32 ...
    longitude       (feature_id) float32 ...
Data variables:
    crs             |S1 ...
    order           (feature_id) int32 ...
    elevation       (feature_id) float32 ...
    streamflow      (feature_id) float64 ...
    q_lateral       (feature_id) float64 ...
    velocity        (feature_id) float64 ...
    qSfcLatRunoff   (feature_id) float64 ...
    qBucket         (feature_id) float64 ...
    qBtmVertRunoff  (feature_id) float64 ...
Attributes: (12/18)
    TITLE:                      OUTPUT FROM WRF-Hydro v5.2.0-beta2
    featureType:                timeSeries
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    model_initialization_time:  1979-02-01_00:00:00
    station_dimension:          feature_id
    model_output_valid_time:    1979-02-02_00:00:00
    ...                         ...
    model_configuration:        retrospective
    dev_OVRTSWCRT:              1
    dev_NOAH_TIMESTEP:          3600
    dev_channel_only:           0
    dev_channelBucket_only:     0
    dev:                        dev_ prefix indicates development/internal me...

xarray chrtout: ds.info:


<xarray.Dataset>
Dimensions:     (feature_id: 2776738, time: 367439)
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
  * feature_id  (feature_id) int32 101 179 181 ... 1180001803 1180001804
    gage_id     (feature_id) |S15 dask.array<chunksize=(2776738,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(2776738,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Data variables:
    crs         |S1 ...
    streamflow  (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    velocity    (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

zarr chrtout: ds.info:


...

xarray chrtout: ds[velocity]:



<xarray.DataArray 'velocity' (time: 367439, feature_id: 2776738)>
dask.array<open_dataset-5ee07aa6121226b4f24157a04a9941efvelocity, shape=(367439, 2776738), dtype=float64, chunksize=(672, 30000), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
  * feature_id  (feature_id) int32 101 179 181 ... 1180001803 1180001804
    gage_id     (feature_id) |S15 dask.array<chunksize=(2776738,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(2776738,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Attributes:
    grid_mapping:  crs
    long_name:     River Velocity
    units:         m s-1


zarr chrtout:  dz[velocity].info:


Name,/velocity
Type,zarr.core.Array
Data type,int32
Shape,"(367439, 2776738)"
Chunk shape,"(672, 30000)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,4081127335928 (3.7T)
No. bytes stored,679656250705 (633.0G)


Un-Compressed Chunk size in MB: 76.904296875
   Compressed Chunk size in MB: 12.8173828125


## precip file: preciptation forcing file

In [8]:
report('precip')

Output file report: /glade/p/datashare/jamesmcc/nwm_retro_v2.1/precip.zarr
Total file size: 2.0T

Dataset overviews:
xarray precip SOURCE single file: ds.info


<xarray.Dataset>
Dimensions:         (time: 1, reference_time: 1, x: 4608, y: 3840)
Coordinates:
  * time            (time) datetime64[ns] 1979-02-02
  * reference_time  (reference_time) datetime64[ns] 1979-02-01
  * x               (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y               (y) float64 -1.92e+06 -1.919e+06 ... 1.918e+06 1.919e+06
Data variables:
    crs             |S1 ...
    U2D             (time, y, x) float64 ...
    V2D             (time, y, x) float64 ...
    LWDOWN          (time, y, x) float64 ...
    RAINRATE        (time, y, x) float32 ...
    T2D             (time, y, x) float64 ...
    Q2D             (time, y, x) float64 ...
    PSFC            (time, y, x) float64 ...
    SWDOWN          (time, y, x) float64 ...
    LQFRAC          (time, y, x) float64 ...
Attributes:
    model_output_valid_time:    1979-02-02_00:00:00
    model_initialization_time:  1979-02-01_00:00:00
    NWM_version_number:         v2.1
    model_configuration:        AORC
    model_output_type:          forcing
    model_total_valid_times:    672.0

xarray precip: ds.info:


,Array,Chunk
Bytes,23.65 TiB,314.03 MiB
Shape,"(367440, 3840, 4608)","(672, 350, 350)"
Count,84239 Tasks,84238 Chunks
Type,float32,numpy.ndarray


zarr precip: ds.info:


...

xarray precip: ds[y]:



<xarray.DataArray 'y' (y: 3840)>
array([-1919500.375, -1918500.375, -1917500.375, ...,  1917499.625,
        1918499.625,  1919499.625])
Coordinates:
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
Attributes:
    _CoordinateAxisType:  GeoY
    long_name:            y coordinate of projection
    resolution:           1000.0
    standard_name:        projection_y_coordinate
    units:                m


zarr precip:  dz[y].info:


Name,/y
Type,zarr.core.Array
Data type,float64
Shape,"(3840,)"
Chunk shape,"(3840,)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,30720 (30.0K)
No. bytes stored,5679 (5.5K)


Un-Compressed Chunk size in MB: 0.029296875
   Compressed Chunk size in MB: 0.005425347222222222


## ldasout file: NoahMP land surface model output

In [9]:
# report('ldasout')

## rtout: overland and subsruface model routing output

In [10]:
# report('rtout')